In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

In [2]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from keras.applications import vgg19
from scipy.optimize import fmin_l_bfgs_b
import imageio
import time

Using TensorFlow backend.


In [3]:
target_image_path = 'content.jpg'
style_reference_image_path = 'style.jpg'
width, height = load_img(target_image_path).size
img_height = 300
img_width = int(width * img_height / height)

In [4]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [5]:
def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [7]:
target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))
input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0)
model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
#model = vgg19.VGG19(input_tensor=input_tensor, weights='vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False)

In [8]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [9]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [10]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [11]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [12]:
def total_variation_loss(x):
    a = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
    b = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, :img_height-1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [13]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

content_layer = 'block5_conv2'
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0]
combination_features = layer_features[2]
loss = loss + content_weight * content_loss(target_image_features, combination_features)

for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1]
    combination_features = layer_features[2]
    loss = loss + (style_weight / len(style_layers)) * style_loss(style_reference_features, combination_features)

loss = loss + total_variation_weight * total_variation_loss(combination_image)    

In [14]:
grads = K.gradients(loss, combination_image)[0]
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [ ]:
result_prefix = 'my_result'
iterations = 100

x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    if i % 10 == 0:
        img = x.copy().reshape((img_height, img_width, 3))
        img = deprocess_image(img)
        fname = result_prefix + '_at_iteration_%d.png' % i
        imageio.imwrite(fname, img)
        print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 8684605000.0
Image saved as my_result_at_iteration_0.png
Iteration 0 completed in 32s
Start of iteration 1
Current loss value: 3224381000.0
Iteration 1 completed in 16s
Start of iteration 2
Current loss value: 2111206000.0
Iteration 2 completed in 17s
Start of iteration 3
Current loss value: 1629824800.0
Iteration 3 completed in 16s
Start of iteration 4
Current loss value: 1354007000.0
Iteration 4 completed in 16s
Start of iteration 5
Current loss value: 1132006500.0
Iteration 5 completed in 16s
Start of iteration 6
Current loss value: 981013300.0
Iteration 6 completed in 15s
Start of iteration 7
Current loss value: 864307100.0
Iteration 7 completed in 15s
Start of iteration 8
Current loss value: 783737540.0
Iteration 8 completed in 16s
Start of iteration 9
Current loss value: 706285600.0
Iteration 9 completed in 16s
Start of iteration 10
Current loss value: 644119550.0
Image saved as my_result_at_iteration_10.png
Iteration 10 completed in 16s
S